In [1]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd
from langfuse.callback import CallbackHandler
import os

load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [2]:
MODEL = "gpt-3.5-turbo"

OUTPUT_FOLDER = "../generated_GUIs/"

IMAGE_INPUT_FOLDER = "../data/"

CREATION_PROMPT = PromptTemplate(
    input_variables=["summary"],
    template= """create the HTML code for a Mobilepage, 
    with a bit of styling, that represents following functionality: 
    {summary}."""
)

SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summaries"],
    template="""Given a collection of Screen summaries, which describe the same screen:
        {summaries}.
        
        Extract the provided information of the screen and formulate a new description. make sure, that no information is lost"""
)

S2W = pd.read_csv("../data/Screen2Words.csv")
S2W = S2W.drop('Unnamed: 0', axis=1)

In [23]:
#function for Creating a GUI based on a given description
def create_gui(id, summary, prompt=CREATION_PROMPT, temp=0):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)

    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    print(response)
    with open(OUTPUT_FOLDER + f"{id}.html", 'w') as f:
        f.write(response)


In [14]:
DESCRIPTION = "This is a flight booking screen from a travel app where users can search for flights, specifying departure and arrival cities, travel dates, number of passengers, and class of service. It also offers options for round trip or one-way flights and displays a section for recent searches"

create_gui(377, DESCRIPTION)

<!DOCTYPE html>
<html>
<head>
    <title>Flight Booking</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f2f2f2;
        }
        
        .container {
            max-width: 600px;
            margin: 0 auto;
            padding: 20px;
            background-color: #fff;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        
        h1 {
            text-align: center;
            margin-bottom: 20px;
        }
        
        label {
            display: block;
            margin-bottom: 10px;
            font-weight: bold;
        }
        
        input[type="text"],
        select {
            width: 100%;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 4px;
            box-sizing: border-box;
            margin-bottom: 20px;
        }
        
        .search-button {
            width: 100%;
            padding: 10px;
      